In [2]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


from tensorflow.keras import models, Sequential, layers, regularizers, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory, get_file
import tensorflow.keras.applications.resnet50 as resnet50
import tensorflow.keras.applications.inception_v3 as inception_v3

from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input


In [3]:
path_train = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/train'
path_test = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/test'


In [18]:
# Function for plotting the loss and accuracy 
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

# Dataset

In [4]:
# # TRAIN DATASET
# train_ds = image_dataset_from_directory(directory=path_train,
#                                       labels='inferred',
#                                       label_mode="categorical",
#                                       validation_split=None,
#                                       subset=None,
#                                       seed=123,
#                                       image_size=(224, 224),
#                                       batch_size=32)   

# # Preprocess X in the train_dataset
# # train_ds_prepro = train_dataset.map(preprocess)

# ###########
# VALIDATION DATASET
# validation_ds = image_dataset_from_directory(directory=path_train,
#                                       labels='inferred',
#                                       label_mode="categorical",
#                                       validation_split=0.2,
#                                       subset="validation",
#                                       seed=123,
#                                       image_size=(224, 224),
#                                       batch_size=32)
                                         
# # Preprocess X in the val_dataset
# # validation_ds_prepro = val_dataset.map(preprocess)

# ############
# # TEST DATASET
# test_ds = image_dataset_from_directory(directory=path_test,
#                                             labels='inferred',
#                                             label_mode="categorical",
#                                             validation_split=None,
#                                             subset=None,
#                                             seed=123,
#                                             image_size=(224, 224),
#                                             batch_size=32) 

# # Preprocess X in the test_dataset
# # test_ds_prepro = test_dataset.map(preprocess)

Found 12000 files belonging to 120 classes.
Found 8580 files belonging to 120 classes.


In [6]:
# # BATCH DATA 
# image_batch_train, labels_batch_train = next(iter(train_ds))
# print(f"train batch: {image_batch_train.shape}, labels: {labels_batch_train.shape}")

# image_batch_val, labels_batch_val = next(iter(validation_ds))
# print(f"validation batch: {image_batch_val.shape}, labels: {labels_batch_val.shape}")

train batch: (32, 224, 224, 3), labels: (32, 120)
validation batch: (32, 224, 224, 3), labels: (32, 120)


In [11]:
# train_ds = train_ds.unbatch()
# images_train = list(train_ds.map(lambda x, y: x))
# labels_train = list(train_ds.map(lambda x, y: y))

# validation_ds = validation_ds.unbatch()
# images_val = list(validation_ds.map(lambda x, y: x))
# labels_val = list(validation_ds.map(lambda x, y: y))

# print(f"From train dataset: {len(labels_train), len(images_train)}")
# print(f"From validation dataset: {len(labels_val), len(images_val)}")

From train dataset: (12000, 12000)
From validation dataset: (2400, 2400)


# Load pre-trained ResNet50 and Inception_V3 models 

In [4]:
# Load the ResNet50 model pre-trained on ImageNet
input_resnet50 = layers.Input(shape=(224,224,3))
input_resnet50 = resnet_preprocess_input(input_resnet50)   #add a preprocessing layer
base_model_resnet50 = resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_resnet50)
# Freeze the base model layers to prevent them from being updated during training
base_model_resnet50.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [5]:
# Load the Inception_V3 model pre-trained on ImageNet
input_inceptionv3 = layers.Input(shape=(224,224,3))
input_inceptionv3 = resnet_preprocess_input(input_inceptionv3)   #add a preprocessing layer
base_model_inceptionv3 = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_tensor= input_inceptionv3)

# Freeze the base model layers to prevent them from being updated during training
base_model_inceptionv3.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


# Combine the pre-trained layers and create the combined model

In [6]:
# Get the output from Resnet50
input_resnet50 = base_model_resnet50.input
output_resnet50 = base_model_resnet50.output
output_resnet50 = layers.Flatten()(output_resnet50)
print(f"The shapes of input and output from Resnet50: {input_resnet50.shape, output_resnet50.shape}")

# Get the input and output from Inception_V3
input_inceptionv3 = base_model_inceptionv3.input
output_inceptionv3 = base_model_inceptionv3.output
output_inceptionv3 = layers.Flatten()(output_inceptionv3)

print(f"The shapes of input and output from Inception_V3: {input_inceptionv3.shape, output_inceptionv3.shape}")

The shapes of input and output from Resnet50: (TensorShape([None, 224, 224, 3]), TensorShape([None, 100352]))
The shapes of input and output from Inception_V3: (TensorShape([None, 224, 224, 3]), TensorShape([None, 51200]))


In [7]:
# Combine the inputs and outputs
combined_inputs = [input_resnet50, input_inceptionv3] 

combined_output = layers.concatenate([output_resnet50, output_inceptionv3])
print(f"combined_output's shape: {combined_output.shape}")


# Add dense layers 
x = layers.Dense(100, activation="relu")(combined_output)
x = layers.Dense(100, activation="relu")(x)

# Add a prediction layer
pred = layers.Dense(120, activation="softmax")(x)

# Initialize the combined model
combined_model = Model(inputs=combined_inputs, outputs=[pred])

# Compile
opt = optimizers.Adam(learning_rate = 1e-4)
combined_model.compile(optimizer=opt, 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

combined_model.summary()

combined_output's shape: (None, 151552)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 224, 224, 3)  0          ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.nn.bias_add (TFOpLambda)    (None, 224, 224, 3)  0           ['tf.__operators__.getitem[0][0]'
                                                      

# Train and evaluate the combined model

In [8]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# Functions for loading data, creating X and y, and encoding y
def load_images(path:str):
    '''
    Load images from the Dog Dataset of Kaggle.
    Returns X and y as numpy arrays
    '''
    X = []
    y = []
    # Use os.walk to iterate over all the subdirectories and files within the given directory
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Extract the label from the path
            # remove .split("-")[1] if you want to keep the ID
            label = os.path.basename(os.path.normpath(subdir)).split("-")[1]
            # Load the image using OpenCV
            img_path = os.path.join(subdir, file)
            img = cv2.imread(img_path)
            # Add the image data and label to the X and y arrays
            X.append(img)
            y.append(label)
    # Convert X and y to NumPy arrays and return th
    return np.array(X), np.array(y)


def load_images_and_preprocess(path):
    '''
    Load images from the Dog Dataset of Kaggle.
    Returns X and y as numpy arrays
    Encode labels as categorical variables
    '''
    X, y = load_images(path)
# assert X.shape == (12000, 224, 224, 3), f"Expected X to have shape (12000, 224, 224, 3), but got {X.shape}. Please check that you provided the correct path and that all images are loaded correctly."
    l_e = LabelEncoder()
    y = l_e.fit_transform(y)
    y = to_categorical(y)
    return X, y

In [9]:
X_train, y_train = load_images_and_preprocess(path_train)
X_train.shape, y_train.shape

((12000, 224, 224, 3), (12000, 120))

In [10]:
# Create X and y from the test dataset
X_test, y_test = load_images_and_preprocess(path_test)
X_test.shape, y_test.shape

((8580, 224, 224, 3), (8580, 120))

In [16]:
# Considering only 1/10th of the 50_000 images
reduction_factor = 2

# Choosing the random indices of small train set and small test set
idx_train =  np.random.choice(len(X_train), round(len(X_train)/reduction_factor), replace=False)
idx_test =  np.random.choice(len(X_test), round(len(X_test)/reduction_factor), replace=False)

# Collecting the two subsamples images_train_small and images_test_small from images_train and images_test
X_train_small = X_train[idx_train]
X_test_small = X_test[idx_test]
# and their corresponding labels
y_train_small = y_train[idx_train]
y_test_small = y_test[idx_test]

print("------------------ Before -----------------")
print(X_train.shape, X_test.shape)

print("")

print("--- After applying the reduction factor ---")
print(X_train_small.shape, X_test_small.shape)

print("")
print("-"*43)

unique, counts = np.unique(y_train_small, return_counts=True)
dict(zip(unique, counts))

------------------ Before -----------------
(12000, 224, 224, 3) (8580, 224, 224, 3)

--- After applying the reduction factor ---
(6000, 224, 224, 3) (4290, 224, 224, 3)

-------------------------------------------


{0.0: 714000, 1.0: 6000}

In [17]:
# Train the combined model 
MODEL = "combined_model"

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=1, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor = 0.1,
                       patience=3,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}.h5".format(MODEL),
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)



history = combined_model.fit(x=[X_train_small, X_train_small],
                             y=y_train_small,
                             validation_split=0.2, 
                             shuffle=True,
                             epochs=5,
                             callbacks=[es, lr, mcp],
                             batch_size=32,
                             verbose=1)

Epoch 1/5
150/150 [==============================] - 40s 170ms/step - loss: 5.5091 - accuracy: 0.0085 - val_loss: 4.7876 - val_accuracy: 0.0067 - lr: 1.0000e-04
Epoch 2/5
150/150 [==============================] - 22s 144ms/step - loss: 4.7844 - accuracy: 0.0121 - val_loss: 4.8168 - val_accuracy: 0.0083 - lr: 1.0000e-04
Epoch 2: early stopping


In [20]:
# Plot the loss and accuracy
plot_history(combined_model, title="")

In [ ]:
# Evaluate the combined model on the test dataset
res = combined_model.evaluate(preprocessed_test_dataset)
test_accuracy = res[-1]
print(f"test_accuracy_model_1 = {round(test_accuracy,2)*100} %")
print(f'Chance level: {1./120*100:.1f}%')